In [1]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.2/435.2 KB 5.2 MB/s eta 0:00:00


In [2]:
!pip install networkx==2.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556009 sha256=d67cef5cb1d41d4aee2f2dcf12c92489f1fff247a277f8b0dc70a6049b14d3ce
  Stored in directory: /root/.cache/pip/wheels/ff/62/9e/0ed2d25fd4f5761e2d19568cda0c32716556dfa682e65ecf64
Successfully built networkx
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


In [3]:
# importing all necessary libraries
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import os
import networkx as nx
import stellargraph as sg
from stellargraph import StellarGraph
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
#from node2vec import Node2Vec as n2v
sns.set()

%matplotlib inline

In [4]:
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GAT

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, feature_extraction, model_selection
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv('Student-Employability-Dataset.csv')
df.head()
# Dataset is now stored in a Pandas Dataframe

,GENERAL APPEARANCE,MANNER OF SPEAKING,PHYSICAL CONDITION,MENTAL ALERTNESS,SELF-CONFIDENCE,ABILITY TO PRESENT IDEAS,COMMUNICATION SKILLS,Student Performance Rating,CLASS
0,4,5,4,5,5,5,5,5,Employable
1,4,4,4,4,4,4,3,5,Employable
2,4,3,3,3,3,3,2,5,LessEmployable
3,3,3,3,2,3,3,3,5,LessEmployable
4,4,4,3,3,4,4,3,5,Employable


In [6]:
df.shape

(2982, 9)

In [7]:
df['CLASS'].value_counts()

Employable        1729
LessEmployable    1253
Name: CLASS, dtype: int64

In [8]:
z = {'Employable' : 1, 'LessEmployable' : 0}
df['CLASS'] = df['CLASS'].map(z)
df.head()

,GENERAL APPEARANCE,MANNER OF SPEAKING,PHYSICAL CONDITION,MENTAL ALERTNESS,SELF-CONFIDENCE,ABILITY TO PRESENT IDEAS,COMMUNICATION SKILLS,Student Performance Rating,CLASS
0,4,5,4,5,5,5,5,5,1
1,4,4,4,4,4,4,3,5,1
2,4,3,3,3,3,3,2,5,0
3,3,3,3,2,3,3,3,5,0
4,4,4,3,3,4,4,3,5,1


In [9]:
#node_Class
node_Class = df["CLASS"]

In [10]:
u3=df.drop(['CLASS'], axis=1)
u3.head()

,GENERAL APPEARANCE,MANNER OF SPEAKING,PHYSICAL CONDITION,MENTAL ALERTNESS,SELF-CONFIDENCE,ABILITY TO PRESENT IDEAS,COMMUNICATION SKILLS,Student Performance Rating
0,4,5,4,5,5,5,5,5
1,4,4,4,4,4,4,3,5
2,4,3,3,3,3,3,2,5
3,3,3,3,2,3,3,3,5
4,4,4,3,3,4,4,3,5


### Distance calculation for creating adjacency matrix


In [11]:
from scipy.spatial.distance import cdist
s = cdist(u3, u3, 'cosine')
s

array([[0.00000000e+00, 1.18196406e-02, 3.89451766e-02, ...,
        5.70574961e-03, 1.75078592e-02, 1.18196406e-02],
       [1.18196406e-02, 0.00000000e+00, 1.07846501e-02, ...,
        6.69736532e-03, 3.57110435e-03, 7.69230769e-03],
       [3.89451766e-02, 1.07846501e-02, 0.00000000e+00, ...,
        2.63323597e-02, 1.15390226e-02, 2.00296533e-02],
       ...,
       [5.70574961e-03, 6.69736532e-03, 2.63323597e-02, ...,
        0.00000000e+00, 1.50356688e-02, 6.69736532e-03],
       [1.75078592e-02, 3.57110435e-03, 1.15390226e-02, ...,
        1.50356688e-02, 1.11022302e-16, 1.14792702e-02],
       [1.18196406e-02, 7.69230769e-03, 2.00296533e-02, ...,
        6.69736532e-03, 1.14792702e-02, 0.00000000e+00]])

In [12]:
X1=pd.DataFrame(s)
X1

,0,1,2,3,4,5,6,7,8,9,...,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981
0,0.000000,0.011820,0.038945,0.031784,0.015827,0.023852,0.023828,0.036119,0.005732,0.010824,...,0.055234,0.021551,0.015827,0.022710,0.031191,0.038945,0.005318,0.005706,1.750786e-02,0.011820
1,0.011820,0.000000,0.010785,0.018050,0.006520,0.010005,0.017776,0.017689,0.003403,0.011479,...,0.039031,0.008924,0.006520,0.006520,0.018689,0.010785,0.007692,0.006697,3.571104e-03,0.007692
2,0.038945,0.010785,0.000000,0.016536,0.011513,0.008478,0.038122,0.009505,0.018376,0.021043,...,0.021836,0.019694,0.011513,0.011513,0.018915,0.000000,0.029275,0.026332,1.153902e-02,0.020030
3,0.031784,0.018050,0.016536,0.000000,0.011438,0.010986,0.043910,0.031092,0.015332,0.020186,...,0.030511,0.023098,0.021630,0.021630,0.043591,0.016536,0.018050,0.030187,1.028900e-02,0.018050
4,0.015827,0.006520,0.011513,0.011438,0.000000,0.007125,0.027587,0.021789,0.008436,0.012128,...,0.034596,0.015815,0.008621,0.017241,0.025511,0.011513,0.014663,0.015583,3.756706e-03,0.014663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,0.038945,0.010785,0.000000,0.016536,0.011513,0.008478,0.038122,0.009505,0.018376,0.021043,...,0.021836,0.019694,0.011513,0.011513,0.018915,0.000000,0.029275,0.026332,1.153902e-02,0.020030
2978,0.005318,0.007692,0.029275,0.018050,0.014663,0.018689,0.026871,0.034339,0.003403,0.011479,...,0.048837,0.017695,0.014663,0.014663,0.036057,0.029275,0.000000,0.006697,1.147927e-02,0.007692
2979,0.005706,0.006697,0.026332,0.030187,0.015583,0.013821,0.017178,0.024151,0.004957,0.007793,...,0.039112,0.020072,0.008125,0.008125,0.021774,0.026332,0.006697,0.000000,1.503567e-02,0.006697
2980,0.017508,0.003571,0.011539,0.010289,0.003757,0.009008,0.018263,0.024357,0.006251,0.016260,...,0.042308,0.008163,0.012128,0.012128,0.026863,0.011539,0.011479,0.015036,1.110223e-16,0.011479


In [13]:
# taking average of the hamming distance matrix
y = np.average(X1)
y

0.01856620671593297

In [14]:
adjmat_df=X1.copy()

In [15]:
# setting threshold value as mean of hamming matrix
adjmat_df[adjmat_df < y]=0
adjmat_df[adjmat_df >= y]=1

In [16]:
# adjacency matrix
adjmat_df

,0,1,2,3,4,5,6,7,8,9,...,2972,2973,2974,2975,2976,2977,2978,2979,2980,2981
0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
2978,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2979,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
G = nx.from_numpy_matrix(np.array(adjmat_df))
G

In [18]:
A=nx.to_numpy_matrix(G)
##Creating graph from adjacency matrix
H=nx.from_numpy_matrix(A)

In [19]:
A=nx.to_numpy_matrix(G)
A

matrix([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 1., 0., 1.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]])

In [20]:
H=nx.from_numpy_matrix(A)
H

### Construct a graph using stellargraph

In [21]:
square_feature_dataframe = StellarGraph.from_networkx(H, node_features=adjmat_df)
print(square_feature_dataframe.info())

StellarGraph: Undirected multigraph
 Nodes: 2982, Edges: 1793526

 Node types:
  default: [2982]
    Features: float32 vector, length 2982
    Edge types: default-default->default

 Edge types:
    default-default->default: [1793526]
        Weights: all 1 (default)
        Features: none


In [22]:
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    ClusterNodeGenerator,
)
from stellargraph import StellarGraph
from stellargraph.layer import GCN, DeepGraphInfomax, GraphSAGE, GAT, APPNP, HinSAGE

from stellargraph import datasets
from stellargraph.utils import plot_history

import pandas as pd
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from IPython.display import display, HTML

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import Model

### Creating the GAT model in Keras
To feed data from the graph to the Keras model we need a generator. Since GAT is a full-batch model, we use the FullBatchNodeGenerator class to feed node features and graph adjacency matrix to the model.

In [23]:
generator = FullBatchNodeGenerator(square_feature_dataframe, method="gat")
generator 

### Converting label to numeric arrays
For our categorical target, we will use one-hot vectors that will be fed into a soft-max Keras layer during training.

In [24]:
target_encoding = preprocessing.LabelBinarizer()

In [25]:
target = target_encoding.fit_transform(df['CLASS'])

In [26]:
target

array([[1],
       [1],
       [0],
       ...,
       [1],
       [0],
       [1]])

### For training we map only the training nodes returned from our splitter and the target values.

In [27]:
gen = generator.flow(df['CLASS'].index, target)
gen

In [28]:
target.shape[1]

1

In [29]:
df['CLASS'].shape

(2982,)

In [30]:
df['CLASS'].astype

<bound method NDFrame.astype of 0       1
1       1
2       0
3       0
4       1
       ..
2977    1
2978    1
2979    1
2980    0
2981    1
Name: CLASS, Length: 2982, dtype: int64>

### Build a GAT model

In [31]:
gat = GAT(
    layer_sizes=[4, target.shape[1]],
    activations=["elu", "softmax"],
    attn_heads=8,
    generator=generator,
    in_dropout=0.5,
    attn_dropout=0.5,
    normalize=None,
)

### The input and output tensors of the GAT model for node prediction, via GAT.in_out_tensors() method

In [32]:
x_inp, predictions = gat.in_out_tensors()

In [33]:
x_inp

[<KerasTensor: shape=(1, 2982, 2982) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(1, None) dtype=int32 (created by layer 'input_2')>,
 <KerasTensor: shape=(1, None, 2) dtype=int64 (created by layer 'input_3')>,
 <KerasTensor: shape=(1, None) dtype=float32 (created by layer 'input_4')>]

In [34]:
predictions

<KerasTensor: shape=(1, None, 1) dtype=float32 (created by layer 'lambda')>

### Training the model

In [35]:
model = Model(inputs=x_inp, outputs=predictions)

In [36]:
model

### Node embeddings

In [37]:
all_nodes = df['CLASS'].index
all_gen = generator.flow(all_nodes)

In [38]:
all_nodes

RangeIndex(start=0, stop=2982, step=1)

In [39]:
all_gen

In [40]:
emb_layer = next(l for l in model.layers if l.name.startswith("graph_attention"))
print(
    "Embedding layer: {}, output shape {}".format(emb_layer.name, emb_layer.output_shape)
)

Embedding layer: graph_attention_sparse, output shape (1, 2982, 32)


In [41]:
emb_layer

In [42]:
embedding_model = Model(inputs=x_inp, outputs=emb_layer.output)

In [43]:
emb = embedding_model.predict(all_gen)
emb.shape

1/1 [==============================] - 4s 4s/step


(1, 2982, 32)

In [44]:
emb

array([[[-0.38527673,  0.7153157 , -0.9036149 , ...,  0.7272412 ,
         -0.55312836,  0.22020592],
        [-0.64035845,  0.74972385, -0.91031325, ...,  1.1608593 ,
         -0.73762417,  0.20866662],
        [-0.6473155 ,  0.8373362 , -0.8501607 , ...,  0.13752405,
         -0.22161078,  0.24726176],
        ...,
        [-0.45787203,  0.70361954, -0.91637975, ...,  0.94050956,
         -0.63040173,  0.4087095 ],
        [-0.6883161 ,  0.6834643 , -0.9079898 , ...,  0.8884238 ,
         -0.6615547 , -0.00774807],
        [-0.5273545 ,  0.8289879 , -0.9106792 , ...,  1.0198805 ,
         -0.66448426,  0.19411992]]], dtype=float32)

In [45]:
 w = emb.reshape(2982, 32)
 w

array([[-0.38527673,  0.7153157 , -0.9036149 , ...,  0.7272412 ,
        -0.55312836,  0.22020592],
       [-0.64035845,  0.74972385, -0.91031325, ...,  1.1608593 ,
        -0.73762417,  0.20866662],
       [-0.6473155 ,  0.8373362 , -0.8501607 , ...,  0.13752405,
        -0.22161078,  0.24726176],
       ...,
       [-0.45787203,  0.70361954, -0.91637975, ...,  0.94050956,
        -0.63040173,  0.4087095 ],
       [-0.6883161 ,  0.6834643 , -0.9079898 , ...,  0.8884238 ,
        -0.6615547 , -0.00774807],
       [-0.5273545 ,  0.8289879 , -0.9106792 , ...,  1.0198805 ,
        -0.66448426,  0.19411992]], dtype=float32)

In [46]:
node_emb = pd.DataFrame(w)
node_emb

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.385277,0.715316,-0.903615,0.569597,0.909326,-0.480350,0.475065,-0.019441,0.418944,-0.123706,...,-0.044206,-0.804639,-0.452501,-0.267746,0.036515,1.109712,0.807389,0.727241,-0.553128,0.220206
1,-0.640358,0.749724,-0.910313,0.686850,1.114133,-0.356935,0.788133,-0.531714,-0.031345,0.336387,...,-0.040397,-0.860080,-0.401332,-0.566356,0.075470,2.273831,1.206020,1.160859,-0.737624,0.208667
2,-0.647316,0.837336,-0.850161,0.681651,0.988778,-0.215474,0.255723,-0.216033,0.283634,0.167947,...,0.047163,-0.673583,-0.304413,0.049260,0.138724,0.610933,0.632354,0.137524,-0.221611,0.247262
3,-0.608834,0.732547,-0.804777,0.554463,0.863906,-0.189742,0.245152,-0.108997,0.356811,0.020870,...,0.060307,-0.641565,-0.319507,0.006202,0.187010,0.604423,0.662682,0.254368,-0.304812,0.137939
4,-0.659649,0.760624,-0.893684,0.631897,0.917104,-0.302396,0.654995,-0.399514,0.248018,-0.049401,...,0.060611,-0.750389,-0.442866,-0.217587,0.173289,1.165948,0.865764,0.612923,-0.465146,0.184582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,-0.647316,0.837336,-0.850161,0.681651,0.988778,-0.215474,0.255723,-0.216033,0.283634,0.167947,...,0.047163,-0.673583,-0.304413,0.049260,0.138724,0.610933,0.632354,0.137524,-0.221611,0.247262
2978,-0.399966,0.615459,-0.883519,0.576336,0.942799,-0.358130,0.683987,-0.330077,0.406368,-0.014734,...,-0.079929,-0.803864,-0.570279,-0.290033,0.077163,1.054254,0.930060,0.877765,-0.569388,0.242267
2979,-0.457872,0.703620,-0.916380,0.609820,1.026346,-0.505827,0.731779,-0.271467,0.422501,-0.043889,...,-0.007538,-0.828837,-0.513248,-0.318615,-0.106112,1.358014,0.941849,0.940510,-0.630402,0.408709
2980,-0.688316,0.683464,-0.907990,0.609243,0.993373,-0.345146,0.805562,-0.460633,0.015701,0.135131,...,0.100457,-0.830017,-0.555280,-0.403823,0.242766,1.490108,1.113169,0.888424,-0.661555,-0.007748


In [47]:
node_emb['Class']=node_Class

In [48]:
node_emb

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,Class
0,-0.385277,0.715316,-0.903615,0.569597,0.909326,-0.480350,0.475065,-0.019441,0.418944,-0.123706,...,-0.804639,-0.452501,-0.267746,0.036515,1.109712,0.807389,0.727241,-0.553128,0.220206,1
1,-0.640358,0.749724,-0.910313,0.686850,1.114133,-0.356935,0.788133,-0.531714,-0.031345,0.336387,...,-0.860080,-0.401332,-0.566356,0.075470,2.273831,1.206020,1.160859,-0.737624,0.208667,1
2,-0.647316,0.837336,-0.850161,0.681651,0.988778,-0.215474,0.255723,-0.216033,0.283634,0.167947,...,-0.673583,-0.304413,0.049260,0.138724,0.610933,0.632354,0.137524,-0.221611,0.247262,0
3,-0.608834,0.732547,-0.804777,0.554463,0.863906,-0.189742,0.245152,-0.108997,0.356811,0.020870,...,-0.641565,-0.319507,0.006202,0.187010,0.604423,0.662682,0.254368,-0.304812,0.137939,0
4,-0.659649,0.760624,-0.893684,0.631897,0.917104,-0.302396,0.654995,-0.399514,0.248018,-0.049401,...,-0.750389,-0.442866,-0.217587,0.173289,1.165948,0.865764,0.612923,-0.465146,0.184582,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,-0.647316,0.837336,-0.850161,0.681651,0.988778,-0.215474,0.255723,-0.216033,0.283634,0.167947,...,-0.673583,-0.304413,0.049260,0.138724,0.610933,0.632354,0.137524,-0.221611,0.247262,1
2978,-0.399966,0.615459,-0.883519,0.576336,0.942799,-0.358130,0.683987,-0.330077,0.406368,-0.014734,...,-0.803864,-0.570279,-0.290033,0.077163,1.054254,0.930060,0.877765,-0.569388,0.242267,1
2979,-0.457872,0.703620,-0.916380,0.609820,1.026346,-0.505827,0.731779,-0.271467,0.422501,-0.043889,...,-0.828837,-0.513248,-0.318615,-0.106112,1.358014,0.941849,0.940510,-0.630402,0.408709,1
2980,-0.688316,0.683464,-0.907990,0.609243,0.993373,-0.345146,0.805562,-0.460633,0.015701,0.135131,...,-0.830017,-0.555280,-0.403823,0.242766,1.490108,1.113169,0.888424,-0.661555,-0.007748,0


In [49]:
X=node_emb.drop(['Class'],axis=1)
Y=node_emb['Class']

### Model Building

### 10 FOLD CROSS VALIDATION

In [50]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import classification_report,accuracy_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import confusion_matrix, recall_score, precision_score,f1_score
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [51]:
from sklearn.svm import LinearSVC
models = LinearSVC()
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
y_pred = []
y_exp = []

for train_index, test_index in folds.split(X, Y):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    models.fit(x_train, y_train)
    # store result from classification
    y_pred.extend(models.predict(x_test))
    # store expected result for this specific fold
    y_exp.extend(y_test)

print("Accuracy: ",  accuracy_score(y_exp, y_pred))
print(classification_report(y_exp, y_pred))
print("Confusion matrix: ", confusion_matrix(y_exp, y_pred))

Accuracy:  0.7126089872568746
              precision    recall  f1-score   support

           0       0.70      0.56      0.62      1253
           1       0.72      0.82      0.77      1729

    accuracy                           0.71      2982
   macro avg       0.71      0.69      0.69      2982
weighted avg       0.71      0.71      0.71      2982

Confusion matrix:  [[ 703  550]
 [ 307 1422]]


In [52]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=10000) 
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
y_pred = []
y_exp = []

for train_index, test_index in folds.split(X, Y):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    lr.fit(x_train, y_train)
    # store result from classification
    y_pred.extend(lr.predict(x_test))
    # store expected result for this specific fold
    y_exp.extend(y_test)

print("Accuracy: ",  accuracy_score(y_exp, y_pred))
print(classification_report(y_exp, y_pred))
print("Confusion matrix: ", confusion_matrix(y_exp, y_pred))

Accuracy:  0.6713615023474179
              precision    recall  f1-score   support

           0       0.65      0.47      0.55      1253
           1       0.68      0.82      0.74      1729

    accuracy                           0.67      2982
   macro avg       0.67      0.64      0.64      2982
weighted avg       0.67      0.67      0.66      2982

Confusion matrix:  [[ 589  664]
 [ 316 1413]]


In [53]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
rf = RandomForestClassifier(n_estimators=300)
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
y_pred = []
y_exp = []

for train_index, test_index in folds.split(X, Y):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    rf.fit(x_train, y_train)
    # store result from classification
    y_pred.extend(rf.predict(x_test))
    # store expected result for this specific fold
    y_exp.extend(y_test)

print("Accuracy: ",  accuracy_score(y_exp, y_pred))
print(classification_report(y_exp, y_pred))
print("Confusion matrix: ", confusion_matrix(y_exp, y_pred))

Accuracy:  0.9071093226022804
              precision    recall  f1-score   support

           0       0.91      0.87      0.89      1253
           1       0.91      0.94      0.92      1729

    accuracy                           0.91      2982
   macro avg       0.91      0.90      0.90      2982
weighted avg       0.91      0.91      0.91      2982

Confusion matrix:  [[1086  167]
 [ 110 1619]]


In [54]:
gbc = GradientBoostingClassifier(n_estimators=300)
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
y_pred = []
y_exp = []

for train_index, test_index in folds.split(X, Y):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    gbc.fit(x_train, y_train)
    # store result from classification
    y_pred.extend(gbc.predict(x_test))
    # store expected result for this specific fold
    y_exp.extend(y_test)

print("Accuracy: ",  accuracy_score(y_exp, y_pred))
print(classification_report(y_exp, y_pred))
print("Confusion matrix: ", confusion_matrix(y_exp, y_pred))

Accuracy:  0.903420523138833
              precision    recall  f1-score   support

           0       0.91      0.86      0.88      1253
           1       0.90      0.93      0.92      1729

    accuracy                           0.90      2982
   macro avg       0.90      0.90      0.90      2982
weighted avg       0.90      0.90      0.90      2982

Confusion matrix:  [[1078  175]
 [ 113 1616]]


In [55]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, 
              gamma=0,  importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, 
             n_estimators=200, n_jobs=16,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, subsample=1,
              tree_method='exact', use_label_encoder=False
              )
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
y_pred = []
y_exp = []

for train_index, test_index in folds.split(X, Y):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    xgbc.fit(x_train, y_train)
    # store result from classification
    y_pred.extend(xgbc.predict(x_test))
    # store expected result for this specific fold
    y_exp.extend(y_test)

print("Accuracy: ",  accuracy_score(y_exp, y_pred))
print(classification_report(y_exp, y_pred))
print("Confusion matrix: ", confusion_matrix(y_exp, y_pred))

Accuracy:  0.9004024144869215
              precision    recall  f1-score   support

           0       0.90      0.86      0.88      1253
           1       0.90      0.93      0.92      1729

    accuracy                           0.90      2982
   macro avg       0.90      0.90      0.90      2982
weighted avg       0.90      0.90      0.90      2982

Confusion matrix:  [[1082  171]
 [ 126 1603]]


In [56]:
import lightgbm as lgb
lgb = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=400, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
y_pred = []
y_exp = []

for train_index, test_index in folds.split(X, Y):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    lgb.fit(x_train, y_train)
    # store result from classification
    y_pred.extend(lgb.predict(x_test))
    # store expected result for this specific fold
    y_exp.extend(y_test)

print("Accuracy: ",  accuracy_score(y_exp, y_pred))
print(classification_report(y_exp, y_pred))
print("Confusion matrix: ", confusion_matrix(y_exp, y_pred))

Accuracy:  0.8839704896042925
              precision    recall  f1-score   support

           0       0.87      0.85      0.86      1253
           1       0.89      0.91      0.90      1729

    accuracy                           0.88      2982
   macro avg       0.88      0.88      0.88      2982
weighted avg       0.88      0.88      0.88      2982

Confusion matrix:  [[1069  184]
 [ 162 1567]]


### TRAIN TEST SPLIT

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7, stratify=Y)

In [58]:
from sklearn.linear_model import PassiveAggressiveClassifier
pa=PassiveAggressiveClassifier(max_iter=1000, random_state=7, tol=1e-3)
pa.fit(x_train, y_train)
y_pred=pa.predict(x_test)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy_score: ', accuracy_score(y_test, y_pred))

[[245   6]
 [283  63]]
              precision    recall  f1-score   support

           0       0.46      0.98      0.63       251
           1       0.91      0.18      0.30       346

    accuracy                           0.52       597
   macro avg       0.69      0.58      0.47       597
weighted avg       0.72      0.52      0.44       597

accuracy_score:  0.5159128978224455


In [59]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC()
lsvm.fit(x_train, y_train)
y_pred=lsvm.predict(x_test)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy_score: ', accuracy_score(y_test, y_pred))

[[136 115]
 [ 60 286]]
              precision    recall  f1-score   support

           0       0.69      0.54      0.61       251
           1       0.71      0.83      0.77       346

    accuracy                           0.71       597
   macro avg       0.70      0.68      0.69       597
weighted avg       0.71      0.71      0.70       597

accuracy_score:  0.7068676716917923


In [60]:
rf = RandomForestClassifier(n_estimators=300)
rf.fit(x_train, y_train)
y_pred=rf.predict(x_test)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy_score: ', accuracy_score(y_test, y_pred))

[[221  30]
 [ 24 322]]
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       251
           1       0.91      0.93      0.92       346

    accuracy                           0.91       597
   macro avg       0.91      0.91      0.91       597
weighted avg       0.91      0.91      0.91       597

accuracy_score:  0.9095477386934674


In [61]:
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, 
              gamma=0,  importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, 
             n_estimators=200, n_jobs=16,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, subsample=1,
              tree_method='exact', use_label_encoder=False
              )
xgbc.fit(x_train, y_train)
y_pred=xgbc.predict(x_test)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy_score: ', accuracy_score(y_test, y_pred))

[[219  32]
 [ 23 323]]
              precision    recall  f1-score   support

           0       0.90      0.87      0.89       251
           1       0.91      0.93      0.92       346

    accuracy                           0.91       597
   macro avg       0.91      0.90      0.90       597
weighted avg       0.91      0.91      0.91       597

accuracy_score:  0.9078726968174204


In [62]:
gbc = GradientBoostingClassifier(n_estimators=300)
gbc.fit(x_train, y_train)
y_pred=gbc.predict(x_test)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy_score: ', accuracy_score(y_test, y_pred))

[[216  35]
 [ 25 321]]
              precision    recall  f1-score   support

           0       0.90      0.86      0.88       251
           1       0.90      0.93      0.91       346

    accuracy                           0.90       597
   macro avg       0.90      0.89      0.90       597
weighted avg       0.90      0.90      0.90       597

accuracy_score:  0.8994974874371859


In [63]:
log = LogisticRegression(max_iter=10000) 
log.fit(x_train, y_train)
y_pred=log.predict(x_test)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy_score: ', accuracy_score(y_test, y_pred))

[[111 140]
 [ 54 292]]
              precision    recall  f1-score   support

           0       0.67      0.44      0.53       251
           1       0.68      0.84      0.75       346

    accuracy                           0.68       597
   macro avg       0.67      0.64      0.64       597
weighted avg       0.67      0.68      0.66       597

accuracy_score:  0.6750418760469011


In [64]:
import lightgbm as lgb
lgb = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=400, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
lgb.fit(x_train, y_train)
y_pred=lgb.predict(x_test)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy_score: ', accuracy_score(y_test, y_pred))

[[215  36]
 [ 30 316]]
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       251
           1       0.90      0.91      0.91       346

    accuracy                           0.89       597
   macro avg       0.89      0.88      0.89       597
weighted avg       0.89      0.89      0.89       597

accuracy_score:  0.8894472361809045
